In [1]:
%reload_ext autoreload
%autoreload 2
%cd ..

/Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor


In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

In [3]:
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')

In [4]:
df.head(5)

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0


In [5]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0 ,0),
    target_column_name='target_rides_next_hour'
)

In [6]:
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 675)
y_train.shape=(32595,)
X_test.shape=(56710, 675)
y_test.shape=(56710,)


In [7]:
import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        """
        Declare a blank function since we do not need to fit the
        model. This is just a placeholder so that we follow standard
        ML frameworks.
        """
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """
        Predict that the rides next hour will be exactly the same
        as the previous hour
        """
        return X_test['rides_previous_1_hour']

In [8]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)

In [9]:
predictions

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
56705    3.0
56706    8.0
56707    8.0
56708    2.0
56709    7.0
Name: rides_previous_1_hour, Length: 56710, dtype: float32

In [10]:
from sklearn.metrics import mean_absolute_error

In [11]:
test_mae = mean_absolute_error(y_test, predictions)

In [12]:
print(f'{test_mae=:.4f}')

test_mae=6.0558


In [13]:
class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t -7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """
        Gets data from t - 7 and returns an array of predictions.
        """
        return X_test[f'rides_previous_{7*24}_hour']

In [14]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [15]:
test_mae = mean_absolute_error(y_test, predictions)

In [16]:
print(f'{test_mae=:.4f}')

test_mae=3.6811


In [17]:
class BaselineModelLast4Weeks:
    """
    Prediction - actual demand observed at t - 7 days, t - 14 days, t - 21 days, t  - 28 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test:pd.DataFrame) -> pd.Series:
        """
        Return predictions for last 4 weeks data
        """
        return 0.25*(
            X_test[f'rides_previous_{1*7*24}_hour'] + \
            X_test[f'rides_previous_{2*7*24}_hour'] + \
            X_test[f'rides_previous_{3*7*24}_hour'] + \
            X_test[f'rides_previous_{4*7*24}_hour']
        )

In [18]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

In [19]:
test_mae = mean_absolute_error(y_test, predictions)

In [20]:
print(f'{test_mae=:.4f}')

test_mae=3.1963
